World Development Indicators - worldbank.org <br/>
Fabio Cardoso - Dec/2021 <br/><br/>

Datasets: <br/><br/>

Country Indicators: <br/>
https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators <br/>
Note: dataset updated with the topics presented in https://data.worldbank.org/indicator?tab=all - Utils notebook. <br/><br/>

Country Codes ISO 3166: <br/>
https://www.iban.com/country-codes <br/><br/>

Note: <br/>
For some orphans indicators, the topic 'private-sector' was assumed - Utils notebook. <br/>
. Firms visited or required meetings with tax officials (% of firms) <br/>
. Net ODA provided to the least developed countries (% of GNI) <br/>
. Net ODA provided, to the least developed countries (current US )−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(−𝑁𝑒𝑡𝑂𝐷𝐴𝑝𝑟𝑜𝑣𝑖𝑑𝑒𝑑,𝑡𝑜𝑡𝑎𝑙(𝑐𝑜𝑛𝑠𝑡𝑎𝑛𝑡2015𝑈𝑆) <br/>
. Net ODA provided, total (current US$)

In [1]:
# Parameters

cloud = True

In [2]:
# Imports

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import zlib
import pickle
from threading import Thread
from multiprocessing import Pool
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

!pip install pyreadstat
import pyreadstat

In [ ]:
# The code was removed by Watson Studio for sharing.

In [4]:
# Storage instantiation

if cloud:
    import types
    import pandas as pd
    import ibm_boto3
    from botocore.client import Config

    def __iter__(self): return 0

    cnx_fcsinsights = ibm_boto3.client(
        service_name = 's3',
        ibm_api_key_id = parm_ibm_api_key_id,
        ibm_service_instance_id = parm_ibm_service_instance_id,
        ibm_auth_endpoint = parm_ibm_auth_endpoint,
        config = Config(signature_version='oauth'),
        endpoint_url = parm_endpoint_url
    )

bkt_in = parm_bkt_in
bkt_out = parm_bkt_out

key_in = 'WDI treated-step4.csv.gz'
key_out = 'WDI treated-step5-general.sav'

In [5]:
# Read data

if cloud:
    cnx_fcsinsights.download_file(Bucket=bkt_in, Key=key_in, Filename=key_in)
    df = pd.read_csv(key_in, compression='gzip')
else:
    df = pd.read_csv('F:/WDI Working Files/'+key_in, compression='gzip')

In [6]:
# Two-way

df['Indicator_Year'] = df[['Indicator Code','Year']].apply(lambda r: r['Indicator Code'] + '_' + str(r['Year']), axis=1)

In [7]:
# Pivoteando

df = df.pivot_table(index='Country Code', columns='Indicator_Year', values='Value')

In [8]:
# Neither all countries have all indicator series. We adopt -1 for null values in a way contries with few series stay close each other in the clustering processing.

df.replace(np.nan, -1, inplace=True)

In [9]:
# Remove spaces from column names
# Note: Pyreadstat does not save index

df.reset_index(inplace=True)

def remove_space_from_cols_names(df):
    rens = dict()
    for col in df.columns:
        if col.find(" "):
            col2 = col.replace(" ", "_")
            rens.update({col:col2})
    return df.rename(columns=rens)

df = remove_space_from_cols_names(df)

In [10]:
# Save intermediate result

if cloud:
    pyreadstat.write_sav(df, key_out)
    cnx_fcsinsights.upload_file(Bucket=bkt_out, Key=key_out, Filename=key_out)
else:
    pyreadstat.write_sav(df, 'F:/WDI Working Files/'+key_out)

In [11]:
print('ok')

ok
